# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from the file `Oders.csv` located in the `data` folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders = pd.read_csv('../data/Orders.csv')
orders.head(3)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
amount_spent_agg = orders.groupby(['CustomerID'])[['amount_spent']].sum()
amount_spent_agg.head(3)

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24


In [4]:
amount_spent_agg['cus_status'] = pd.qcut(amount_spent_agg['amount_spent'], q=[0, .75, .95, 1], labels=['normal', 'preferred', 'VIP'])
amount_spent_agg.rename(columns={'amount_spent':'total_amount_spent'}, inplace=True)
amount_spent_agg.head(3)

,total_amount_spent,cus_status
CustomerID,,
12346,77183.60,VIP
12347,4310.00,preferred
12348,1797.24,preferred


In [8]:
orders_final = pd.merge(orders, amount_spent_agg, how = 'right', on = 'CustomerID')
orders_final.head(3)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,total_amount_spent,cus_status
0,61619,541431,23166,2011,1,2,10,medium ceramic top storage jar,74215,2011-01-18 10:01:00,1.04,12346,United Kingdom,77183.6,77183.6,VIP
1,14938,537626,85116,2010,12,2,14,black candelabra t-light holder,12,2010-12-07 14:57:00,2.10,12347,Iceland,25.2,4310.0,preferred
2,14939,537626,22375,2010,12,2,14,airline bag vintage jet set brown,4,2010-12-07 14:57:00,4.25,12347,Iceland,17.0,4310.0,preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [23]:
orders_country = orders_final.groupby(['Country'])[['cus_status']]
orders_country

In [30]:
VIP_df = orders_final[orders_final['cus_status'].str.contains('VIP')].groupby('Country').size()
VIP_df.sort_values(ascending=False)

Country
United Kingdom     84185
EIRE                7077
France              3290
Germany             3127
Netherlands         2080
Australia            898
Portugal             681
Switzerland          594
Spain                511
Norway               420
Channel Islands      364
Finland              294
Cyprus               248
Singapore            222
Japan                205
Sweden               198
Belgium               54
Denmark               36
dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [31]:
VIP_preferred_df = orders_final[orders_final['cus_status'].str.contains('VIP|preferred')].groupby('Country').size()
VIP_preferred_df.sort_values(ascending=False)

Country
United Kingdom     221635
Germany              7349
EIRE                 7238
France               6301
Netherlands          2080
Spain                1569
Belgium              1557
Switzerland          1370
Portugal             1093
Australia            1028
Norway               1028
Channel Islands       589
Italy                 507
Finland               504
Cyprus                451
Sweden                273
Japan                 272
Singapore             222
Denmark               217
Israel                214
Iceland               182
Austria               158
Poland                149
Canada                135
Malta                  67
Lebanon                45
Greece                 32
dtype: int64